In [ ]:
import pandas as pd
import numpy as np

In [ ]:
import json
import csv

with open('dev-v2.0.json') as json_file:
	data = json.load(json_file)

data

In [ ]:
def generate_texts_queries_answers(path):
    with open(path, 'rb') as f:
        squad_dict = json.load(f)
    texts, queries, answers = [], [], []
    for group in squad_dict['data']:
        for passage in group['paragraphs']:
            context = passage['context']
            for qa in passage['qas']:
                question = qa['question']
                for answer in qa['answers']:
                    texts.append(context)
                    queries.append(question)
                    answers.append(answer)
    return texts, queries, answers


In [ ]:
texts, queries, answers = generate_texts_queries_answers('dev-v2.0.json')

In [ ]:
queries

In [ ]:
answers_text = [item['text'] for item in answers]

In [ ]:
answers_text

In [ ]:
datadf = pd.DataFrame({'context': texts, 'Questions': queries, 'answers': answers_text})
print(datadf)


In [ ]:
datadf

In [ ]:
datadf = datadf.drop_duplicates()

In [ ]:

datadf.to_csv('sample.csv',index = False)


## Loading dataset

In [ ]:
data = pd.read_csv("/kaggle/input/graduation/sample.csv")
data

# Question Generation not good in generation because it needs answer as input
hugging face : https://huggingface.co/mrm8488/t5-base-finetuned-question-generation-ap

In [ ]:
from transformers import AutoModelWithLMHead, AutoTokenizer

tokenizer_t5base = AutoTokenizer.from_pretrained("mrm8488/t5-base-finetuned-question-generation-ap")
model_t5base = AutoModelWithLMHead.from_pretrained("mrm8488/t5-base-finetuned-question-generation-ap")


In [ ]:
def get_question(answer, context, max_length=64):
    input_text = "answer: %s  context: %s </s>" % (answer, context)
    features = tokenizer_t5base([input_text], return_tensors='pt')
    output = model_t5base.generate(input_ids=features['input_ids'], 
               attention_mask=features['attention_mask'],
               max_length=max_length)

    return tokenizer_t5base.decode(output[0])

context = "Manuel has created RuPERTa-base with the support of HF-Transformers and Google"
answer = "Manuel"

get_question(answer, context)


## T5

In [ ]:
from transformers import T5Tokenizer, T5ForConditionalGeneration

tokenizer = T5Tokenizer.from_pretrained("t5-3b")
model = T5ForConditionalGeneration.from_pretrained("t5-3b")

In [ ]:
t5_questions_results = []
def get_question_t5(context):
    text = "context: %s. Generate Question  "%context
    input_ids = tokenizer(text, return_tensors="pt")
    generate = model.generate(**input_ids)
    t5_questions_results.append(tokenizer.decode(generate[0]))
data["context"].sample(n=10).apply(lambda x : get_question_t5(x))
t5_questions_results

## Bert

hugging_face link: https://huggingface.co/voidful/context-only-question-generator

In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained("voidful/context-only-question-generator")
model = AutoModelForSeq2SeqLM.from_pretrained("voidful/context-only-question-generator")

In [ ]:
def generate_question_using_context_only(context):
    inputs = tokenizer(context, return_tensors="pt")
    outputs = model.generate(inputs["input_ids"])
    return tokenizer.decode(outputs[0]) 

context = "byden is the american president"
generate_question_using_context_only(context)

# t5 trained on squad
model on hugging face:
https://huggingface.co/p208p2002/bart-squad-qg-hl?text=Harry+Potter+is+a+series+of+seven+fantasy+novels+written+by+British+author%2C+%5BHL%5DJ.+K.+Rowling%5BHL%5D.


In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
  
tokenizer_t5squad = AutoTokenizer.from_pretrained("p208p2002/t5-squad-qg-hl", use_fast=False)

model_t5squad = AutoModelForSeq2SeqLM.from_pretrained("p208p2002/t5-squad-qg-hl")

In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import pandas as pd

# Load T5 model and tokenizer
tokenizer_t5squad = AutoTokenizer.from_pretrained("p208p2002/t5-squad-qg-hl", use_fast=False)
model_t5squad = AutoModelForSeq2SeqLM.from_pretrained("p208p2002/t5-squad-qg-hl")

# Initialize an empty list to store generated outputs
generated_outputs = []

# Loop through each context in the dataframe
for input_text in datadf["context"]:
    input_ids = tokenizer_t5squad.encode(input_text, return_tensors="pt")
    output = model_t5squad.generate(input_ids, max_length=64)
    generated_text = tokenizer_t5squad.batch_decode(output)[0]
    generated_outputs.append(generated_text)


In [ ]:


from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import pandas as pd

# Load T5 model and tokenizer
tokenizer_t5squad = AutoTokenizer.from_pretrained("p208p2002/t5-squad-qg-hl", use_fast=False)
model_t5squad = AutoModelForSeq2SeqLM.from_pretrained("p208p2002/t5-squad-qg-hl")

# Initialize an empty list to store generated outputs
generated_outputs = []

# Loop through each context in the dataframe
for input_text in datadf["context"]:
    input_ids = tokenizer_t5squad.encode(input_text, return_tensors="pt")
    output = model_t5squad.generate(input_ids, max_length=64)
    generated_text = tokenizer_t5squad.batch_decode(output)[0]
    generated_outputs.append(generated_text)


In [ ]:
!pip install sentencepiece
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

# Load T5 model and tokenizer
tokenizer_t5squad = AutoTokenizer.from_pretrained("p208p2002/t5-squad-qg-hl", use_fast=False)
model_t5squad = AutoModelForSeq2SeqLM.from_pretrained("p208p2002/t5-squad-qg-hl")

# List to store generated outputs
generated_outputs_t5 = []

# Loop through the DataFrame and generate questions using T5
for context_text in datadf["context"]:
    # Tokenize and encode the input text
    input_ids = tokenizer_t5squad.encode(context_text, return_tensors="pt", max_length=512, truncation=True)

    # Generate questions using T5
    output_ids = model_t5squad.generate(input_ids, max_length=64)
    generated_text = tokenizer_t5squad.batch_decode(output_ids, skip_special_tokens=True)[0]
    
    # Append generated text to the list
    generated_outputs_t5.append(generated_text)

# Display the generated outputs
for context, generated_text in zip(datadf["context"], generated_outputs_t5):
    print(f"Context: {context}")
    print(f"Generated Question: {generated_text}\n")


# bert for question answering
hugging face link: https://huggingface.co/docs/transformers/v4.37.2/en/model_doc/bert#transformers.BertForQuestionAnswering

In [ ]:
from transformers import AutoTokenizer, BertForQuestionAnswering
import torch
tokenizer_bert = AutoTokenizer.from_pretrained("deepset/bert-base-cased-squad2")
model_bert = BertForQuestionAnswering.from_pretrained("deepset/bert-base-cased-squad2")

In [ ]:
for _, row in datadf.iterrows():
    question = row['Questions']
    text = row['context']

    inputs = tokenizer_bert(question, text, return_tensors="pt")
    with torch.no_grad():
        outputs = model_bert(**inputs)

    answer_start_index = outputs.start_logits.argmax()
    answer_end_index = outputs.end_logits.argmax()

    predict_answer_tokens = inputs.input_ids[0, answer_start_index: answer_end_index + 1]
    predicted_answer = tokenizer_bert.decode(predict_answer_tokens, skip_special_tokens=True)

    answers.append(predicted_answer)

# PREDICTION


In [ ]:
from transformers import AutoTokenizer, BertForQuestionAnswering
import torch

tokenizer_bert = AutoTokenizer.from_pretrained("deepset/bert-base-cased-squad2")
model_bert = BertForQuestionAnswering.from_pretrained("deepset/bert-base-cased-squad2")

# Load the sample data
datadf_sample = pd.read_csv('sample.csv')

# Get the last two rows
last_two_rows = datadf_sample.sample(n=2)

for _, row in last_two_rows.iterrows():
    question = row['Questions']
    text = row['context']

    inputs = tokenizer_bert(question, text, return_tensors="pt")
    with torch.no_grad():
        outputs = model_bert(**inputs)

    answer_start_index = outputs.start_logits.argmax()
    answer_end_index = outputs.end_logits.argmax()

    predict_answer_tokens = inputs.input_ids[0, answer_start_index: answer_end_index + 1]
    predicted_answer = tokenizer_bert.decode(predict_answer_tokens, skip_special_tokens=True)

    print(predicted_answer)


In [ ]:
from transformers import AutoTokenizer, BertForQuestionAnswering
import torch
import pandas as pd

tokenizer_bert = AutoTokenizer.from_pretrained("deepset/bert-base-cased-squad2")
model_bert = BertForQuestionAnswering.from_pretrained("deepset/bert-base-cased-squad2")

# Load the sample data
datadf_sample = pd.read_csv('sample.csv')

# Get the last two rows
last_two_rows = datadf_sample.sample(n=2)

for _, row in last_two_rows.iterrows():
    question = row['Questions']
    text = row['context']

    inputs = tokenizer_bert(question, text, return_tensors="pt")
    with torch.no_grad():
        outputs = model_bert(**inputs)

    answer_start_index = outputs.start_logits.argmax()
    answer_end_index = outputs.end_logits.argmax()

    predict_answer_tokens = inputs.input_ids[0, answer_start_index: answer_end_index + 1]
    predicted_answer = tokenizer_bert.decode(predict_answer_tokens, skip_special_tokens=True)

    # Print question and predicted answer
    print(f"Question: {question}")
    print(f"Predicted Answer: {predicted_answer}")
    print()


In [ ]:
from transformers import AutoTokenizer, BertForQuestionAnswering
import torch
import pandas as pd

tokenizer_bert = AutoTokenizer.from_pretrained("deepset/bert-base-cased-squad2")
model_bert = BertForQuestionAnswering.from_pretrained("deepset/bert-base-cased-squad2")

# Load the sample data
datadf_sample = pd.read_csv('sample.csv')

# Get the last two rows
last_two_rows = datadf_sample.sample(n=2)

for _, row in last_two_rows.iterrows():
    question = row['Questions']
    text = row['context']

    inputs = tokenizer_bert(question, text, return_tensors="pt")
    with torch.no_grad():
        outputs = model_bert(**inputs)

    answer_start_index = outputs.start_logits.argmax()
    answer_end_index = outputs.end_logits.argmax()

    predict_answer_tokens = inputs.input_ids[0, answer_start_index: answer_end_index + 1]
    predicted_answer = tokenizer_bert.decode(predict_answer_tokens, skip_special_tokens=True)

    # Print question and predicted answer
    print(f"Question: {question}")
    print(f"Predicted Answer: {predicted_answer}")
    print()


In [ ]:
from transformers import AutoTokenizer, BertForQuestionAnswering
import torch
import pandas as pd

tokenizer_bert = AutoTokenizer.from_pretrained("deepset/bert-base-cased-squad2")
model_bert = BertForQuestionAnswering.from_pretrained("deepset/bert-base-cased-squad2")

# Load the sample data
datadf_sample = pd.read_csv('sample.csv')

# Get the last two rows
last_two_rows = datadf_sample.sample(n=2)

for _, row in last_two_rows.iterrows():
    question = row['Questions']
    text = row['context']

    inputs = tokenizer_bert(question, text, return_tensors="pt")
    with torch.no_grad():
        outputs = model_bert(**inputs)

    answer_start_index = outputs.start_logits.argmax()
    answer_end_index = outputs.end_logits.argmax()

    predict_answer_tokens = inputs.input_ids[0, answer_start_index: answer_end_index + 1]
    predicted_answer = tokenizer_bert.decode(predict_answer_tokens, skip_special_tokens=True)

    # Print question and predicted answer
    print(f"Question: {question}")
    print(f"Predicted Answer: {predicted_answer}")
    print()


In [ ]:
from transformers import AutoTokenizer, BertForQuestionAnswering
import torch
import pandas as pd

tokenizer_bert = AutoTokenizer.from_pretrained("deepset/bert-base-cased-squad2")
model_bert = BertForQuestionAnswering.from_pretrained("deepset/bert-base-cased-squad2")

# Load the sample data
datadf_sample = pd.read_csv('sample.csv')

# Get the last two rows
last_two_rows = datadf_sample.sample(n=2)

for _, row in last_two_rows.iterrows():
    question = row['Questions']
    text = row['context']

    inputs = tokenizer_bert(question, text, return_tensors="pt")
    with torch.no_grad():
        outputs = model_bert(**inputs)

    answer_start_index = outputs.start_logits.argmax()
    answer_end_index = outputs.end_logits.argmax()

    predict_answer_tokens = inputs.input_ids[0, answer_start_index: answer_end_index + 1]
    predicted_answer = tokenizer_bert.decode(predict_answer_tokens, skip_special_tokens=True)

    # Print question and predicted answer
    print(f"Question: {question}")
    print(f"Predicted Answer: {predicted_answer}")
    print()


In [ ]:
from transformers import AutoTokenizer, BertForQuestionAnswering
import torch
import pandas as pd

tokenizer_bert = AutoTokenizer.from_pretrained("deepset/bert-base-cased-squad2")
model_bert = BertForQuestionAnswering.from_pretrained("deepset/bert-base-cased-squad2")

# Load the sample data
datadf_sample = pd.read_csv('sample.csv')

# Get the last two rows
last_two_rows = datadf_sample.sample(n=2)

for _, row in last_two_rows.iterrows():
    question = row['Questions']
    text = row['context']

    inputs = tokenizer_bert(question, text, return_tensors="pt")
    with torch.no_grad():
        outputs = model_bert(**inputs)

    answer_start_index = outputs.start_logits.argmax()
    answer_end_index = outputs.end_logits.argmax()

    predict_answer_tokens = inputs.input_ids[0, answer_start_index: answer_end_index + 1]
    predicted_answer = tokenizer_bert.decode(predict_answer_tokens, skip_special_tokens=True)

    # Print question and predicted answer
    print(f"Question: {question}")
    print(f"Predicted Answer: {predicted_answer}")
    print()


In [ ]:
from transformers import AutoTokenizer, BertForQuestionAnswering
import torch
import pandas as pd

tokenizer_bert = AutoTokenizer.from_pretrained("deepset/bert-base-cased-squad2")
model_bert = BertForQuestionAnswering.from_pretrained("deepset/bert-base-cased-squad2")

# Load the sample data
datadf_sample = pd.read_csv('sample.csv')

# Get the last two rows
last_two_rows = datadf_sample.sample(n=2)

for _, row in last_two_rows.iterrows():
    question = row['Questions']
    text = row['context']

    inputs = tokenizer_bert(question, text, return_tensors="pt")
    with torch.no_grad():
        outputs = model_bert(**inputs)

    answer_start_index = outputs.start_logits.argmax()
    answer_end_index = outputs.end_logits.argmax()

    predict_answer_tokens = inputs.input_ids[0, answer_start_index: answer_end_index + 1]
    predicted_answer = tokenizer_bert.decode(predict_answer_tokens, skip_special_tokens=True)

    # Print question and predicted answer
    print(f"Question: {question}")
    print(f"Predicted Answer: {predicted_answer}")
    print()


In [ ]:
from transformers import AutoTokenizer, BertForQuestionAnswering
import torch
import pandas as pd

tokenizer_bert = AutoTokenizer.from_pretrained("deepset/bert-base-cased-squad2")
model_bert = BertForQuestionAnswering.from_pretrained("deepset/bert-base-cased-squad2")

# Load the sample data
datadf_sample = pd.read_csv('sample.csv')

# Get the last two rows
last_two_rows = datadf_sample.sample(n=2)

for _, row in last_two_rows.iterrows():
    question = row['Questions']
    text = row['context']

    inputs = tokenizer_bert(question, text, return_tensors="pt")
    with torch.no_grad():
        outputs = model_bert(**inputs)

    answer_start_index = outputs.start_logits.argmax()
    answer_end_index = outputs.end_logits.argmax()

    predict_answer_tokens = inputs.input_ids[0, answer_start_index: answer_end_index + 1]
    predicted_answer = tokenizer_bert.decode(predict_answer_tokens, skip_special_tokens=True)

    # Print question and predicted answer
    print(f"Question: {question}")
    print(f"Predicted Answer: {predicted_answer}")
    print()


# *test one*

In [ ]:
!pip install nltk
!pip install rouge


In [ ]:
from nltk.translate.bleu_score import sentence_bleu
from transformers import AutoTokenizer, BertForQuestionAnswering
import torch

tokenizer_bert = AutoTokenizer.from_pretrained("deepset/bert-base-cased-squad2")
model_bert = BertForQuestionAnswering.from_pretrained("deepset/bert-base-cased-squad2")

# Load the sample data
datadf_sample = pd.read_csv('sample.csv')

# Get the last two rows
last_two_rows = datadf_sample.sample(n=2)

# List to store the predicted answers
answers = []

for _, row in last_two_rows.iterrows():
    question = row['Questions']
    text = row['context']

    inputs = tokenizer_bert(question, text, return_tensors="pt")
    with torch.no_grad():
        outputs = model_bert(**inputs)

    answer_start_index = outputs.start_logits.argmax()
    answer_end_index = outputs.end_logits.argmax()

    predict_answer_tokens = inputs.input_ids[0, answer_start_index: answer_end_index + 1]
    predicted_answer = tokenizer_bert.decode(predict_answer_tokens, skip_special_tokens=True)

    # Append the predicted answer to the list
    answers.append(predicted_answer)

# Now, let's compute the BLEU score
references = [[row['answers']] for _, row in last_two_rows.iterrows()]
bleu_scores = [sentence_bleu([ref], pred) for ref, pred in zip(references, answers)]

print(bleu_scores)


In [ ]:
from rouge import Rouge
from transformers import AutoTokenizer, BertForQuestionAnswering
import torch

tokenizer_bert = AutoTokenizer.from_pretrained("deepset/bert-base-cased-squad2")
model_bert = BertForQuestionAnswering.from_pretrained("deepset/bert-base-cased-squad2")

# Load the sample data
datadf_sample = pd.read_csv('sample.csv')

# Get the last two rows
last_two_rows = datadf_sample.sample(n=2)

# List to store the predicted answers
answers = []

# Initialize Rouge scorer
rouge_scorer = Rouge()

for _, row in last_two_rows.iterrows():
    question = row['Questions']
    text = row['context']

    inputs = tokenizer_bert(question, text, return_tensors="pt")
    with torch.no_grad():
        outputs = model_bert(**inputs)

    answer_start_index = outputs.start_logits.argmax()
    answer_end_index = outputs.end_logits.argmax()

    predict_answer_tokens = inputs.input_ids[0, answer_start_index: answer_end_index + 1]
    predicted_answer = tokenizer_bert.decode(predict_answer_tokens, skip_special_tokens=True)

    # Append the predicted answer to the list
    answers.append(predicted_answer)

# Now, let's compute the ROUGE score
references = [[row['answers']] for _, row in last_two_rows.iterrows()]
rouge_scores = [rouge_scorer.get_scores(pred, ref)[0]['rouge-l']['f'] for pred, ref in zip(answers, references)]

print(rouge_scores)


# test 2



In [ ]:
from transformers import AutoTokenizer, BertForQuestionAnswering
import torch
import pandas as pd
from nltk.translate.bleu_score import corpus_bleu

# Load a sample of the data
sample_size = 100  # You can adjust the sample size as needed
sample_data = datadf.sample(n=sample_size, random_state=42)

# Initialize BERT model and tokenizer
tokenizer_bert = AutoTokenizer.from_pretrained("deepset/bert-base-cased-squad2")
model_bert = BertForQuestionAnswering.from_pretrained("deepset/bert-base-cased-squad2")

# Lists to store predicted answers
predicted_answers = []

# Iterate through the sample data
for _, row in sample_data.iterrows():
    question = row['Questions']
    text = row['context']

    inputs = tokenizer_bert(question, text, return_tensors="pt")
    with torch.no_grad():
        outputs = model_bert(**inputs)

    answer_start_index = outputs.start_logits.argmax()
    answer_end_index = outputs.end_logits.argmax()

    predict_answer_tokens = inputs.input_ids[0, answer_start_index: answer_end_index + 1]
    predicted_answer = tokenizer_bert.decode(predict_answer_tokens, skip_special_tokens=True)

    predicted_answers.append(predicted_answer)

# Add predicted answers to the sample data
sample_data['predicted_answers'] = predicted_answers

# Tokenize the reference and predicted texts for BLEU metric
reference_texts = sample_data['answers'].apply(lambda x: [x]).tolist()
predicted_texts = sample_data['predicted_answers'].tolist()

# BLEU metric
bleu_score = corpus_bleu(reference_texts, predicted_texts)
print(f"BLEU Score: {bleu_score}")


In [ ]:
import nltk
nltk.download('wordnet')

In [ ]:
# Download Open Multilingual Wordnet (omw-1.4) dataset
nltk.download('omw-1.4')

In [ ]:
from transformers import AutoTokenizer, BertForQuestionAnswering
import torch
import pandas as pd
from nltk.translate.meteor_score import single_meteor_score
import nltk
# Download WordNet dataset
nltk.download('wordnet')
# Tokenize the reference and predicted texts for METEOR metric
tokenized_reference_texts = [ref.split() for ref in sample_data['answers']]
tokenized_predicted_texts = [pred.split() for pred in sample_data['predicted_answers']]

# METEOR metric
meteor_score = sum(single_meteor_score(ref, pred) for ref, pred in zip(tokenized_reference_texts, tokenized_predicted_texts)) / len(tokenized_predicted_texts)
print(f"METEOR Score: {meteor_score}")


In [ ]:
!pip install rouge


In [ ]:
from transformers import AutoTokenizer, BertForQuestionAnswering
import torch
import pandas as pd
from rouge import Rouge

# Combine tokenized lists into strings for ROUGE metric
reference_texts = [' '.join(ref) for ref in tokenized_reference_texts]
predicted_texts = [' '.join(pred) for pred in tokenized_predicted_texts]

# Initialize ROUGE scorer
rouge = Rouge()

# ROUGE metric
try:
    scores = rouge.get_scores(predicted_texts, reference_texts, avg=True)
    rouge_score = scores['rouge-1']['f']
    print(f"ROUGE Score: {rouge_score}")
except ValueError as e:
    print(f"Error calculating ROUGE: {e}")
